In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
from local_utils.retrieval import get_listening_data

In [2]:
from datetime import datetime, timezone

df = get_listening_data()
start = datetime(2025, 1, 1, tzinfo=timezone.utc)
end = datetime(2025+1, 1, 1, tzinfo=timezone.utc)
mask = (df.listened_at >= start) & (df.listened_at < end)

df = df.loc[mask]
df

,listened_at,track_name,artist_name
86121,2025-01-01 01:07:47+00:00,Europaträume,Brutalismus 3000
86122,2025-01-01 01:10:27+00:00,Can You Feel It?!,Mutilator
86123,2025-01-01 01:15:13+00:00,We Both Suffocate,Snow Strippers
86124,2025-01-01 01:17:56+00:00,Stupid (Can’t run from the urge),underscores
86125,2025-01-01 17:58:08+00:00,Hoe-nouns (feat. Thundercat & reggie),Smino
...,...,...,...
105603,2025-12-17 09:52:29+00:00,No Te Metas - Sassy 009 Remix,Lover's Skit
105604,2025-12-17 09:55:27+00:00,Let Me Say This,Skin On Skin
105605,2025-12-17 09:59:40+00:00,joyride,The Crying Nudes
105606,2025-12-17 10:01:17+00:00,4 Raws,EsDeeKid


aim is to group by week, then by hour of day?

In [3]:
df.groupby([pd.Grouper(key="listened_at", freq="W-MON")])

In [7]:
df_grouped = df.groupby([df['listened_at'].dt.isocalendar().week, df['listened_at'].dt.hour], as_index=False).size()

In [21]:
hourly_by_week = df.groupby([df['listened_at'].dt.isocalendar().week, df['listened_at'].dt.hour]).size().reset_index(name='freq')

# Get all combinations of weeks (1-52) and hours (0-23)
all_weeks = range(1, 53)
all_hours = range(0, 24)
index = pd.MultiIndex.from_product([all_weeks, all_hours], names=['week', 'hour'])

# Reindex to include all combinations, filling missing values with 0
hourly_by_week = hourly_by_week.set_index(['week', 'listened_at']).reindex(index, fill_value=0).reset_index()

hourly_by_week

,week,hour,freq
0,1,0,0
1,1,1,4
2,1,2,0
3,1,3,0
4,1,4,0
...,...,...,...
1243,52,19,0
1244,52,20,0
1245,52,21,0
1246,52,22,0


In [22]:
hourly_by_week["freq"]+5

0       5
1       9
2       5
3       5
4       5
       ..
1243    5
1244    5
1245    5
1246    5
1247    5
Name: freq, Length: 1248, dtype: int64